In [35]:
import io 
import numpy as np 
import random

import tensorflow as tf
from tensorflow import keras
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

import re

# Import Dataset 

The dataset 'divina_commedia.txt' is imported, containing the text of Dante's Divine Comedy. The file includes additional lines that are not part of the original text but provide information about the author.

In [36]:
print('Opening file...')
path = "divina_commedia.txt"
with io.open(path, encoding='utf-8') as file:
    text = file.read().lower()

print("text length", len(text))
print()
print('\n\n\n\n\n', '***** first 1000 characters *****', '\n\n\n\n\n')
text[0:1000]

Opening file...
text length 558240






 ***** first 1000 characters ***** 







"inferno\n\n\n\ninferno: canto i\n\n\nnel mezzo del cammin di nostra vita\n  mi ritrovai per una selva oscura\n  che' la diritta via era smarrita.\n\nahi quanto a dir qual era e` cosa dura\n  esta selva selvaggia e aspra e forte\n  che nel pensier rinova la paura!\n\ntant'e` amara che poco e` piu` morte;\n  ma per trattar del ben ch'i' vi trovai,\n  diro` de l'altre cose ch'i' v'ho scorte.\n\nio non so ben ridir com'i' v'intrai,\n  tant'era pien di sonno a quel punto\n  che la verace via abbandonai.\n\nma poi ch'i' fui al pie` d'un colle giunto,\n  la` dove terminava quella valle\n  che m'avea di paura il cor compunto,\n\nguardai in alto, e vidi le sue spalle\n  vestite gia` de' raggi del pianeta\n  che mena dritto altrui per ogne calle.\n\nallor fu la paura un poco queta\n  che nel lago del cor m'era durata\n  la notte ch'i' passai con tanta pieta.\n\ne come quei che con lena affannata\n  uscito fuor del pelago a la riva\n  si volge a l'acqua perigliosa e guata,\n\ncosi` l'animo mio, 

In [37]:
# Remove information about the author
text = re.sub(r"(e-text courtesy progetto manuzio)", "", text, flags=re.DOTALL)

print("text length", len(text))
print()
print('\n\n\n\n\n', '***** first 1000 characters *****', '\n\n\n\n\n')
text[0:1000]

text length 558176






 ***** first 1000 characters ***** 







"inferno\n\n\n\ninferno: canto i\n\n\nnel mezzo del cammin di nostra vita\n  mi ritrovai per una selva oscura\n  che' la diritta via era smarrita.\n\nahi quanto a dir qual era e` cosa dura\n  esta selva selvaggia e aspra e forte\n  che nel pensier rinova la paura!\n\ntant'e` amara che poco e` piu` morte;\n  ma per trattar del ben ch'i' vi trovai,\n  diro` de l'altre cose ch'i' v'ho scorte.\n\nio non so ben ridir com'i' v'intrai,\n  tant'era pien di sonno a quel punto\n  che la verace via abbandonai.\n\nma poi ch'i' fui al pie` d'un colle giunto,\n  la` dove terminava quella valle\n  che m'avea di paura il cor compunto,\n\nguardai in alto, e vidi le sue spalle\n  vestite gia` de' raggi del pianeta\n  che mena dritto altrui per ogne calle.\n\nallor fu la paura un poco queta\n  che nel lago del cor m'era durata\n  la notte ch'i' passai con tanta pieta.\n\ne come quei che con lena affannata\n  uscito fuor del pelago a la riva\n  si volge a l'acqua perigliosa e guata,\n\ncosi` l'animo mio, 

# Preprocessing data

During the data preprocessing phase, the text is divided into individual *canti*, removing unnecessary titles to retain only the 100 *canti* from the Divine Comedy.

In [38]:
canti = re.split(r'(?<=\n)(inferno|purgatorio|paradiso):\s*canto\s*[ivxlcdm]+\n*', text)

canti = [canto.strip() for canto in canti if canto.strip() and canto.lower() not in ['inferno', 'purgatorio', 'paradiso']]

# The first element of 'cantos', namely 'Inferno', is removed from the list to obtain a list containing only the cantos. 
if canti[0].lower() in ['inferno', 'purgatorio', 'paradiso']:
    canti = canti[1:]

print("Number of canti: ", len(canti))

# Print second canto
print(canti[1]) 

Number of canti:  100
lo giorno se n'andava, e l'aere bruno
  toglieva li animai che sono in terra
  da le fatiche loro; e io sol uno

m'apparecchiava a sostener la guerra
  si` del cammino e si` de la pietate,
  che ritrarra` la mente che non erra.

o muse, o alto ingegno, or m'aiutate;
  o mente che scrivesti cio` ch'io vidi,
  qui si parra` la tua nobilitate.

io cominciai: <<poeta che mi guidi,
  guarda la mia virtu` s'ell'e` possente,
  prima ch'a l'alto passo tu mi fidi.

tu dici che di silvio il parente,
  corruttibile ancora, ad immortale
  secolo ando`, e fu sensibilmente.

pero`, se l'avversario d'ogne male
  cortese i fu, pensando l'alto effetto
  ch'uscir dovea di lui e 'l chi e 'l quale,

non pare indegno ad omo d'intelletto;
  ch'e' fu de l'alma roma e di suo impero
  ne l'empireo ciel per padre eletto:

la quale e 'l quale, a voler dir lo vero,
  fu stabilita per lo loco santo
  u' siede il successor del maggior piero.

per quest'andata onde li dai tu vanto,
  intese cos

In [39]:
chars = sorted(list(set(text)))
print('total chars: ', len(chars))

char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

print(char_indices)
print(indices_char)

total chars:  40
{'\n': 0, ' ': 1, '!': 2, '"': 3, "'": 4, '(': 5, ')': 6, ',': 7, '-': 8, '.': 9, ':': 10, ';': 11, '<': 12, '>': 13, '?': 14, '`': 15, 'a': 16, 'b': 17, 'c': 18, 'd': 19, 'e': 20, 'f': 21, 'g': 22, 'h': 23, 'i': 24, 'j': 25, 'l': 26, 'm': 27, 'n': 28, 'o': 29, 'p': 30, 'q': 31, 'r': 32, 's': 33, 't': 34, 'u': 35, 'v': 36, 'x': 37, 'y': 38, 'z': 39}
{0: '\n', 1: ' ', 2: '!', 3: '"', 4: "'", 5: '(', 6: ')', 7: ',', 8: '-', 9: '.', 10: ':', 11: ';', 12: '<', 13: '>', 14: '?', 15: '`', 16: 'a', 17: 'b', 18: 'c', 19: 'd', 20: 'e', 21: 'f', 22: 'g', 23: 'h', 24: 'i', 25: 'j', 26: 'l', 27: 'm', 28: 'n', 29: 'o', 30: 'p', 31: 'q', 32: 'r', 33: 's', 34: 't', 35: 'u', 36: 'v', 37: 'x', 38: 'y', 39: 'z'}


# Definition of functions

In [40]:
def generete_sequences(text, maxlen, step):
    sentences = []
    next_chars = []

    for i in range(0, len(text) - maxlen, step):
        sentences.append(text[i: i + maxlen])
        next_chars.append(text[i + maxlen])
    
    return sentences, next_chars

In [41]:
def encode_sequences(sentences, next_chars, maxlen, chars, char_indices):
    x = np.zeros((len(sentences), maxlen, len(chars)), dtype=bool)
    y = np.zeros((len(sentences), len(chars)), dtype=bool)

    for i, sentence in enumerate(sentences):
        for t, char in enumerate(sentence):
            x[i, t, char_indices[char]] = 1
        y[i, char_indices[next_chars[i]]] = 1

    return x, y

In [42]:
import matplotlib.pyplot as plt

def plot_performance(history):
    flg, ax = plt.subplots(1,2)
    flg.tight_layout()
    train_acc = history.history['accuracy']
    train_loss = history.history['loss']
    val_acc = history.history['val_accuracy']
    val_loss = history.history['val_loss']
    ax[0].set_xlabel('Epochs')
    ax[0].set_ylabel('Loss')
    ax[0].set_title('Loss')
    ax[0].plot(train_loss, label='Training Loss')
    ax[0].plot(val_loss, label='Validation Loss')
    ax[0].legend()
    
    ax[1].set_xlabel('Epochs')
    ax[1].set_ylabel('Accuracy')
    ax[1].set_title('Accuracy')
    ax[1].plot(train_acc, label='Training Accuracy')
    ax[1].plot(val_acc, label='Validation Accuracy')
    ax[1].legend()

    plt.show()

In [43]:
import sys

def testAfterEpoch(epoch, _, maxlen, text, model):
    print()
    print()
    print('***** Epoch: %d *****' % (epoch+1))
    start_index = random.randint(0, len(text)- maxlen - 1)

    generated = ''
    sentence = text[start_index : start_index + maxlen]
    generated = generated + sentence

    print('***** starting sentence *****') 
    print(sentence)
    print('*****************************')
    sys.stdout.write(generated)

    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1
        
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = np.argmax(preds)
        next_char = indices_char[next_index]

        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

# Divide the sequence into training, validation and test

## Model 1 - Divide the sequence into training, validation and testing in an unbalanced way.

Si divide il dataset in training, validation e test andando a prendere in modo randomico i canti appena divisi. Al training si assegna il 70 percento dei canti, al validation il 20 e 10 per il test. 

Dato che i canti vengono mescolati all'inizo, questo potrebbe influire sui risultati per questo è stato definito che è un modo non bilanciato. 

Per questo primo addestramento è stata mantenuta l'architettura usata durante il laboratorio.  

In [44]:
random.shuffle(canti)

train_size = int(0.7 * len(canti))
val_size = int(0.2 * len(canti))
test_size = len(canti) - train_size - val_size

train_canti = canti[:train_size]
val_canti = canti[train_size:train_size + val_size]
test_canti = canti[train_size + val_size:]

print(f"Training set: {len(train_canti)} canti")
print(f"Validation set: {len(val_canti)} canti")
print(f"Test set: {len(test_canti)} canti")

Training set: 70 canti
Validation set: 20 canti
Test set: 10 canti


In [45]:
maxlen = 30 # chunk length
step = 3

sentences_train, next_chars_train = generete_sequences(''.join(train_canti), maxlen, step)
sentences_val, next_chars_val = generete_sequences(''.join(val_canti), maxlen, step)
sentences_test, next_chars_test = generete_sequences(''.join(test_canti), maxlen, step)

print('number of sequences: ', len(sentences_train))
print(sentences_train[11])
print(next_chars_train[11])

number of sequences:  128555

  giu` nel secondo, che men l
o


In [46]:
x_train, y_train = encode_sequences(sentences_train, next_chars_train, maxlen, chars, char_indices)
x_val, y_val = encode_sequences(sentences_val, next_chars_val, maxlen, chars, char_indices)
x_test, y_test = encode_sequences(sentences_test, next_chars_test, maxlen, chars, char_indices)

In [47]:
print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"x_val shape: {x_val.shape}")
print(f"y_val shape: {y_val.shape}")
print(f"x_test shape: {x_test.shape}")
print(f"y_test shape: {y_test.shape}")

x_train shape: (128555, 30, 40)
y_train shape: (128555, 40)
x_val shape: (37821, 30, 40)
y_val shape: (37821, 40)
x_test shape: (18765, 30, 40)
y_test shape: (18765, 40)


Aggiungiamo all'architettura di rete anche la metrica di accuracy

In [48]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(
    loss='categorical_crossentropy', 
    optimizer=optimizer,
    metrics=['accuracy']
    )
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 128)            │        86,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 40)             │         5,160 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 91,688 (358.16 KB)

 Trainable params: 91,688 (358.16 KB)

 Non-trainable params: 0 (0.00 B)

In [49]:
text_canti = ''.join(train_canti)

print_callback = LambdaCallback(
    on_epoch_end=lambda epoch, logs: testAfterEpoch(
        epoch,
        logs,
        maxlen,
        text_canti,
        model
        )
    )

In [50]:
history = model.fit(x_train, y_train,
        validation_data=(x_val, y_val),
        batch_size = 2048, 
        epochs = 20, 
        callbacks = [print_callback], 
        )

Epoch 1/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 362ms/step - accuracy: 0.1708 - loss: 3.0630

***** Epoch: 1 *****
***** starting sentence *****
i si dice,
  di qua che dire e
*****************************
i si dice,
  di qua che dire e di si si se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se se 
63/63 ━━━━━━━━━━━━━━━━━━━━ 55s 858ms/step - accuracy: 0.1715 - loss: 3.0585 - val_accuracy: 0.2936 - val_loss: 2.3692
Epoch 2/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 363ms/step - accuracy: 0.3088 - loss: 2.2679

***** Epoch: 2 *****
***** starting sentence *****
erso questa rivera,
  tanto ch
*****************************
erso questa rivera,
  tanto che l'alla che l'alla che

In [ ]:
plot_performance(history)

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss', score[0])
print('Test accuracy', score[1])

## Model 2 - Divide the sequence into training, validation and testing in an balanced way.

In [ ]:
# Pre processing dei dati andando a dividere i canti

cantiche = re.split(r'\n(inferno|purgatorio|paradiso)\n', text)

cantiche = [canto.strip() for canto in cantiche if canto.strip() and canto.lower() not in ['inferno', 'purgatorio', 'paradiso']]

print(cantiche)
print("Total cantiche: ", len(cantiche))
print(cantiche[1])

In [ ]:
# INFERNO
canti_inferno = re.split(r'(?<=\n)(inferno):\s*canto\s*[ivxlcdm]+\n*', cantiche[0])

canti_inferno = [canto.strip() for canto in canti_inferno if canto.strip() and canto.lower() not in ['inferno']]

# The first element of 'canti', namely 'Inferno', is removed from the list to obtain a list containing only the canti
if canti_inferno[0].lower() in ['inferno', 'purgatorio', 'paradiso']:
    canti_inferno = canti_inferno[1:]

print("Number of canti: ", len(canti_inferno))

train_size_inferno = int(0.7 * len(canti_inferno))
val_size_inferno = int(0.2 * len(canti_inferno))
test_size_inferno = len(canti_inferno) - train_size_inferno - val_size_inferno

train_inferno = canti_inferno[:train_size_inferno]
val_inferno = canti_inferno[train_size_inferno:train_size_inferno + val_size_inferno]
test_inferno = canti_inferno[train_size_inferno + val_size_inferno:]

print(f"Training set: {len(train_inferno)} canti")
print(f"Validation set: {len(val_inferno)} canti")
print(f"Test set: {len(test_inferno)} canti")

In [ ]:
# PURGATORIO
canti_purgatorio = re.split(r'(?<=\n)(purgatorio):\s*canto\s*[ivxlcdm]+\n*', cantiche[1])

canti_purgatorio = [canto.strip() for canto in canti_purgatorio if canto.strip() and canto.lower() not in ['purgatorio']]

# The first element of 'canti', namely 'Inferno', is removed from the list to obtain a list containing only the canti 
if canti_purgatorio[0].lower() in ['purgatorio']:
    canti_purgatorio = canti_purgatorio[1:]

print("Number of canti: ", len(canti_purgatorio))

train_size_purgatorio = int(0.7 * len(canti_purgatorio))
val_size_purgatorio = int(0.2 * len(canti_purgatorio))
test_size_purgatorio = len(canti_purgatorio) - train_size_purgatorio - val_size_purgatorio

train_purgatorio = canti_purgatorio[:train_size_purgatorio]
val_purgatorio = canti_purgatorio[train_size_purgatorio:train_size_purgatorio + val_size_purgatorio]
test_purgatorio = canti_purgatorio[train_size_purgatorio + val_size_purgatorio:]

print(f"Training set: {len(train_purgatorio)} canti")
print(f"Validation set: {len(val_purgatorio)} canti")
print(f"Test set: {len(test_purgatorio)} canti")

In [ ]:
# PARADISO
canti_paradiso = re.split(r'(?<=\n)(paradiso):\s*canto\s*[ivxlcdm]+\n*', cantiche[2])

canti_paradiso = [canto.strip() for canto in canti_paradiso if canto.strip() and canto.lower() not in ['paradiso']]

# The first element of 'canti', namely 'Inferno', is removed from the list to obtain a list containing only the canti. 
if canti_paradiso[0].lower() in ['paradiso']:
    canti_paradiso = canti_paradiso[1:]

print("Number of canti: ", len(canti_paradiso))

train_size_paradiso = int(0.7 * len(canti_paradiso))
val_size_paradiso = int(0.2 * len(canti_paradiso))
test_size_paradiso = len(canti_paradiso) - train_size_paradiso - val_size_paradiso

train_paradiso = canti_paradiso[:train_size_paradiso]
val_paradiso = canti_paradiso[train_size_paradiso:train_size_paradiso + val_size_paradiso]
test_paradiso = canti_paradiso[train_size_paradiso + val_size_paradiso:]

print(f"Training set: {len(train_paradiso)} canti")
print(f"Validation set: {len(val_paradiso)} canti")
print(f"Test set: {len(test_paradiso)} canti")

In [ ]:
# Uniamo i canti suddivisi per ciascuna cantica nei rispettivi set finali
train_canti = train_inferno + train_purgatorio + train_paradiso
val_canti = val_inferno + val_purgatorio + val_paradiso
test_canti = test_inferno + test_purgatorio + test_paradiso

# Stampa la lunghezza dei set
print(f"Training set: {len(train_canti)} canti")
print(f"Validation set: {len(val_canti)} canti")
print(f"Test set: {len(test_canti)} canti")

In [ ]:
text_for_training = ''.join(train_canti)

In [ ]:
maxlen = 30 # chunk length
step = 3

sentences_train, next_chars_train = generete_sequences(''.join(train_canti), maxlen, step)
sentences_val, next_chars_val = generete_sequences(''.join(val_canti), maxlen, step)
sentences_test, next_chars_test = generete_sequences(''.join(test_canti), maxlen, step)

print('number of sequences: ', len(sentences_train))
print(sentences_train[11])
print(next_chars_train[11])

In [ ]:
x_train, y_train = encode_sequences(sentences_train, next_chars_train, maxlen, chars, char_indices)
x_val, y_val = encode_sequences(sentences_val, next_chars_val, maxlen, chars, char_indices)
x_test, y_test = encode_sequences(sentences_test, next_chars_test, maxlen, chars, char_indices)

print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"x_val shape: {x_val.shape}")
print(f"y_val shape: {y_val.shape}")
print(f"x_test shape: {x_test.shape}")
print(f"y_test shape: {y_test.shape}")

In [ ]:
model_b = Sequential()
model_b.add(LSTM(128, input_shape=(maxlen, len(chars))))
model_b.add(Dense(len(chars), activation='softmax'))

optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.01)
model_b.compile(
    loss='categorical_crossentropy', 
    optimizer=optimizer,
    metrics=['accuracy']
    )
model_b.summary()

In [40]:
print_callback = LambdaCallback(
    on_epoch_end=lambda epoch, logs: testAfterEpoch(
        epoch,
        logs,
        maxlen,
        text_for_training,
        model_b
    )
)

In [ ]:
history_b = model_b.fit(x_train, y_train,
        validation_data=(x_val, y_val),
        batch_size = 2048, 
        epochs = 20, 
        callbacks = [print_callback], 
        )

In [ ]:
plot_performance(history_b)

In [ ]:
score = model_b.evaluate(x_test, y_test, verbose=0)
print('Test loss', score[0])
print('Test accuracy', score[1])

# Tune the chunk length

## Model 3 - maxlen 50

In [ ]:
maxlen = 50 # chunk length
step = 3

sentences_train, next_chars_train = generete_sequences(''.join(train_canti), maxlen, step)
sentences_val, next_chars_val = generete_sequences(''.join(val_canti), maxlen, step)
sentences_test, next_chars_test = generete_sequences(''.join(test_canti), maxlen, step)

print('number of sequences: ', len(sentences_train))
print(sentences_train[11])
print(next_chars_train[11])

In [ ]:
x_train, y_train = encode_sequences(sentences_train, next_chars_train, maxlen, chars, char_indices)
x_val, y_val = encode_sequences(sentences_val, next_chars_val, maxlen, chars, char_indices)
x_test, y_test = encode_sequences(sentences_test, next_chars_test, maxlen, chars, char_indices)

print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"x_val shape: {x_val.shape}")
print(f"y_val shape: {y_val.shape}")
print(f"x_test shape: {x_test.shape}")
print(f"y_test shape: {y_test.shape}")

In [ ]:
model3 = Sequential()
model3.add(LSTM(128, input_shape=(maxlen, len(chars))))
model3.add(Dense(len(chars), activation='softmax'))

optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.01)
model3.compile(
    loss='categorical_crossentropy', 
    optimizer=optimizer,
    metrics=['accuracy']
    )
model3.summary()

In [47]:
print_callback = LambdaCallback(
    on_epoch_end=lambda epoch, logs: testAfterEpoch(
        epoch,
        logs,
        maxlen,
        text_for_training,
        model3
    )
)

In [ ]:
history3 = model3.fit(x_train, y_train,
        validation_data=(x_val, y_val),
        batch_size = 2048, 
        epochs = 20, 
        callbacks = [print_callback], 
        )

In [ ]:
plot_performance(history3)

In [ ]:
score = model3.evaluate(x_test, y_test, verbose=0)
print('Test loss', score[0])
print('Test accuracy', score[1])

## maxlen 20

In [ ]:
maxlen = 20 # chunk length
step = 2

sentences_train, next_chars_train = generete_sequences(''.join(train_canti), maxlen, step)
sentences_val, next_chars_val = generete_sequences(''.join(val_canti), maxlen, step)
sentences_test, next_chars_test = generete_sequences(''.join(test_canti), maxlen, step)

print('number of sequences: ', len(sentences_train))
print(sentences_train[11])
print(next_chars_train[11])

In [ ]:
x_train, y_train = encode_sequences(sentences_train, next_chars_train, maxlen, chars, char_indices)
x_val, y_val = encode_sequences(sentences_val, next_chars_val, maxlen, chars, char_indices)
x_test, y_test = encode_sequences(sentences_test, next_chars_test, maxlen, chars, char_indices)

print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"x_val shape: {x_val.shape}")
print(f"y_val shape: {y_val.shape}")
print(f"x_test shape: {x_test.shape}")
print(f"y_test shape: {y_test.shape}")

In [ ]:
model4 = Sequential()
model4.add(LSTM(128, input_shape=(maxlen, len(chars))))
model4.add(Dense(len(chars), activation='softmax'))

optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.01)
model4.compile(
    loss='categorical_crossentropy', 
    optimizer=optimizer,
    metrics=['accuracy']
    )
model4.summary()

In [54]:
print_callback = LambdaCallback(
    on_epoch_end=lambda epoch, logs: testAfterEpoch(
        epoch,
        logs,
        maxlen,
        text_for_training,
        model4
    )
)

In [ ]:
history4 = model4.fit(x_train, y_train,
        validation_data=(x_val, y_val),
        batch_size = 2048, 
        epochs = 20, 
        callbacks = [print_callback], 
        )

In [ ]:
plot_performance(history4)

In [ ]:
score = model4.evaluate(x_test, y_test, verbose=0)
print('Test loss', score[0])
print('Test accuracy', score[1])

## maxlen 80

In [ ]:
maxlen = 80 # chunk length
step = 3

sentences_train, next_chars_train = generete_sequences(''.join(train_canti), maxlen, step)
sentences_val, next_chars_val = generete_sequences(''.join(val_canti), maxlen, step)
sentences_test, next_chars_test = generete_sequences(''.join(test_canti), maxlen, step)

print('number of sequences: ', len(sentences_train))
print(sentences_train[11])
print(next_chars_train[11])

In [ ]:
x_train, y_train = encode_sequences(sentences_train, next_chars_train, maxlen, chars, char_indices)
x_val, y_val = encode_sequences(sentences_val, next_chars_val, maxlen, chars, char_indices)
x_test, y_test = encode_sequences(sentences_test, next_chars_test, maxlen, chars, char_indices)

print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"x_val shape: {x_val.shape}")
print(f"y_val shape: {y_val.shape}")
print(f"x_test shape: {x_test.shape}")
print(f"y_test shape: {y_test.shape}")

In [ ]:
model5 = Sequential()
model5.add(LSTM(128, input_shape=(maxlen, len(chars))))
model5.add(Dense(len(chars), activation='softmax'))

optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.01)
model5.compile(
    loss='categorical_crossentropy', 
    optimizer=optimizer,
    metrics=['accuracy']
    )
model5.summary()

In [61]:
print_callback = LambdaCallback(
    on_epoch_end=lambda epoch, logs: testAfterEpoch(
        epoch,
        logs,
        maxlen,
        text_for_training,
        model5
    )
)

In [ ]:
history5 = model5.fit(x_train, y_train,
        validation_data=(x_val, y_val),
        batch_size = 2048, 
        epochs = 20, 
        callbacks = [print_callback], 
        )

In [ ]:
plot_performance(history5)

In [ ]:
score = model5.evaluate(x_test, y_test, verbose=0)
print('Test loss', score[0])
print('Test accuracy', score[1])